In [1]:
%load_ext Cython

In [3]:
%%cython

# -*- coding: utf-8 -*-
"""
Created on Sun Nov 12 15:22:49 2017

@author: simone
"""

import numpy as np
import scipy.sparse as sps
import deleteBlanks as db
import timeit
print ("ciao")

train_test_split = 0.80

userList =  np.genfromtxt('./CsvFiles/train_final.csv', dtype = None, skip_header = True, usecols = (0), delimiter = "\t")
itemList = np.genfromtxt('./CsvFiles/train_final.csv', dtype = int, skip_header = True, usecols = (1), delimiter = "\t")
numInteractions = itemList.size
ratingList = [1] * numInteractions

train_mask = np.random.choice([True,False], numInteractions, [train_test_split, 1-train_test_split])
test_mask = np.logical_not(train_mask)


userList = np.array(userList)
itemList = np.array(itemList)
ratingList = np.array(ratingList)

track_dic_aux = np.load('./ContentBasedUtilities/track_dic_aux.npy').item()
dic_track = np.load('./ContentBasedUtilities/dic_track.npy').item()

playlists,tracks,playlist_dic_aux,track_dic_aux,track_dic = db.deleteBlanksPartial(userList[train_mask], itemList[train_mask],{},track_dic_aux,dic_track)
playlists_test,tracks_test,playlist_dic_aux,track_dic_aux,dic_track = db.deleteBlanksPartial(userList[test_mask],itemList[test_mask],playlist_dic_aux,track_dic_aux,dic_track)


playlists = np.array(playlists)
tracks = np.array(tracks)
playlists_test = np.array(playlists_test)
tracks_test = np.array(tracks_test)
data = playlists.size * [1]


tracks_train_mask = np.in1d(tracks,tracks_test, invert = True)
tracks_train = tracks[tracks_train_mask]



URM_train =sps.csr_matrix((data, (playlists, tracks)),shape = (57561,100000))

userList_unique = np.unique(np.concatenate((playlists,playlists_test)))
userList_unique_train = np.unique(playlists)

numberOfInteractionsTest = tracks_test.size
data_test = [1] * numberOfInteractionsTest

URM_test =sps.csr_matrix((data_test, (playlists_test, tracks_test)),shape = (57561,100000))

tracks_train_mask = np.in1d(tracks,tracks_test, invert = True)
tracks = np.array(tracks)
tracks_train = tracks[tracks_train_mask]

loader = np.load('./ContentBasedUtilities/similarity_matrix.csv.npz')


similarity_matrix = sps.csr_matrix((loader['data'], loader['indices'], loader['indptr']),shape = loader['shape'])


real_user_rating = URM_train.dot(similarity_matrix)
real_user_rating = real_user_rating.tocsr()

print ("done real user rating")

def cosine_similarities(mat):
    #mat_norm = pp.normalize(mat.tocsc(), axis=0)
    matrix1= mat.T.tocsr().astype(float)
    matrix2 = mat.tocsr().astype(float)
    simil =  sps.csr_matrix.dot(matrix1,matrix2)
    simil = simil - sps.dia_matrix((simil.diagonal()[np.newaxis, :], [0]), shape=simil.shape)
    return simil

values, rows, cols = [], [], []

def KNN(item_weights,URM_train):
        nitems = item_weights.shape[0]

        i = 0
        for element in range(item_weights.shape[0]):
            #print("Item %d of %d" % (i, item_weights.shape[0]))
            if (i%1000 == 0):
                print("Item %d of %d" % (i, item_weights.shape[0]))
            i = i+1
            this_item_weights = item_weights[element,:].toarray()[0]
            top_k_idx = np.argpartition(this_item_weights, -140)[-140:]
            #top_k_idx = np.argsort(this_item_weights) [-140:]
            values.extend(this_item_weights[top_k_idx])
            rows.extend(np.arange(nitems)[top_k_idx])
            cols.extend(np.ones(140) * element)
        print (max(rows))
        return sps.csc_matrix((values, (rows, cols)), shape=(nitems, nitems), dtype=np.float32)

#URM_train = URM_train.tocsc()

user_similarity = cosine_similarities(URM_train.T)
user_similarity = user_similarity.tocsr()
start = timeit.default_timer()
user_similarity = KNN(user_similarity,URM_train)
print ("execution time knn user similarity: " + str(timeit.default_timer()-start))
print (URM_train.shape, user_similarity.shape)
user_real = user_similarity.dot(URM_train)

values, rows, cols = [], [], []

collaborative_similarity = cosine_similarities(URM_train)
collaborative_similarity = collaborative_similarity.tocsr()
start = timeit.default_timer()
collaborative_similarity = KNN(collaborative_similarity,URM_train)
print ("execution time knn collaborative: " + str(timeit.default_timer()-start))
collaborative_real = URM_train.dot(collaborative_similarity)

final_real_ratings = user_real + user_real + collaborative_real

def recommend(playlist_id):
        # compute the scores using the dot product
        
        scores = (final_real_ratings[playlist_id]).toarray().ravel()
        ranking = scores
        # rank items
        ranking = ranking.argsort()[::-1]        
        ranking = _filter_seen(playlist_id, ranking)
        return ranking[:5]
    
def _filter_seen(user_id, ranking):
        user_profile = URM_train[user_id]
        seen = user_profile.indices
        all_tracks = tracks_train
        all_tracks = np.unique(np.concatenate((seen,tracks_train)))        
        unseen_mask = np.in1d(ranking, all_tracks, assume_unique=True, invert=True)
        return ranking[unseen_mask]

def precision(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision_score

def recall(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall_score

def MAP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    
    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

def evaluate_algorithm(URM_test, at=5):
    
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0
    
    num_eval = 0


    for i,user_id in  enumerate(userList_unique):
        
        if i % 500 == 0:
            print("User %d of %d" % (i, len(userList_unique)))

        relevant_items = URM_test[user_id].indices
        
        if len(relevant_items)>0:
            
            recommended_items = recommend(user_id)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_MAP += MAP(recommended_items, relevant_items)


    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval
    
    print("Recommender performance is: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP))



evaluate_algorithm(URM_test)

ciao


UnicodeError: Unpickling a python object failed: UnicodeDecodeError('ascii', b'\xb0\xaa \x00', 0, 1, 'ordinal not in range(128)')
You may need to pass the encoding= option to numpy.load